In [37]:
import os

In [47]:
%pwd

'C:\\Users\\preet\\OneDrive\\Desktop\\End-to-end text summarization\\Transformer-based_End-to-End_Text_Summarization'

In [48]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_Dir : Path

In [49]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml , create_directories

In [50]:
class configurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig (
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_Dir = config.unzip_Dir
        )

        return data_ingestion_config


In [51]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [52]:
class DataIngestion :
    def __init__(self, config : DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file) :
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_Dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [53]:
try:
    config = configurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-04 17:52:03,160: INFO: common: yaml file: config\config.yaml loaded successfully]


[2025-02-04 17:52:03,209: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-04 17:52:03,214: INFO: common: Created directory at: artifacts]
[2025-02-04 17:52:03,218: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-02-04 17:52:06,519: INFO: 530534375: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4358:E7459:EDA28:158575:67A20666
Accept-Ranges: bytes
Date: Tue, 04 Feb 2025 12:21:59 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100027-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1738671718.409194,VS0,VE978
Vary: Authorization,Accept-Enc